# Import package

In [1]:
import nltk
import pyLDAvis.gensim
import gensim
from gensim import corpora
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

/Users/admin/miniconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /Users/admin/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data set

In [2]:
from nltk.corpus import reuters as corpus

num_of_documents = 100
docs = [corpus.words(fileid) for fileid in corpus.fileids()[:num_of_documents]]

print("Number of documents:", len(docs))

Number of documents: 100


# Preprocessing

In [3]:
en_stop = nltk.corpus.stopwords.words('english')
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [4]:
from nltk.corpus import wordnet as wn

def preprocess_word(word, stopwordset):
    
    word=word.lower()
    
    if word in [",","."]:
        return None
    
    if word in stopwordset:
        return None
    
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: 
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [5]:
pre_docs = preprocess_documents(docs)

print("Before preprocessing:")
print(docs[0][:25])
print("After preprocessing:")
print(pre_docs[0][:25])

Before preprocessing:
['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
After preprocessing:
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


## Change K here

In [24]:
n_topic = 20

# LDA

In [25]:
dictionary = corpora.Dictionary(pre_docs)
corpus_ = [dictionary.doc2bow(doc) for doc in pre_docs]

In [26]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=n_topic,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

In [27]:
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.009*"atlas" + 0.007*"bundesbank" + 0.006*"banks" + 0.006*"marks" + 0.006*"market" + 0.006*"bond" + 0.005*"corp" + 0.005*"would" + 0.005*"loan" + 0.005*"bank" + 0.005*"quarter" + 0.005*"security" + 0.004*"two" + 0.004*"oil" + 0.004*"development"')
(1, '0.009*"first" + 0.006*"export" + 0.006*"debt" + 0.006*"surplus" + 0.005*"corp" + 0.005*"banker" + 0.004*"yen" + 0.004*"korea" + 0.004*"1985" + 0.004*"paxar" + 0.004*"trade" + 0.004*"current" + 0.004*"oil" + 0.004*"dividend" + 0.004*"spokesman"')
(2, '0.006*"market" + 0.006*"1985" + 0.006*"would" + 0.005*"official" + 0.005*"dividend" + 0.005*"japan" + 0.004*"also" + 0.004*"import" + 0.004*"f" + 0.004*"wheat" + 0.004*"vw" + 0.004*"board" + 0.004*"co" + 0.004*"last" + 0.003*"dutch"')
(3, '0.007*"trade" + 0.007*"inc" + 0.006*"qtr" + 0.006*"28" + 0.006*"1st" + 0.006*"canadian" + 0.005*"last" + 0.005*"sales" + 0.005*"22" + 0.005*"ltd" + 0.004*"would" + 0.004*"loss" + 0.004*"co" + 0.004*"japan" + 0.004*"statement"')
(4, '0.015*"stg" + 0.0

In [28]:
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(3, 0.7375541), (13, 0.14931764), (17, 0.05216737), (19, 0.05708209)]
document ID 1:[(10, 0.9666593)]
document ID 2:[(6, 0.5476744), (18, 0.43624756)]
document ID 3:[(11, 0.95199025), (17, 0.028850617)]
document ID 4:[(11, 0.9809937)]
document ID 5:[(17, 0.98318404)]
document ID 6:[(9, 0.9919595)]
document ID 7:[(2, 0.94061804)]
document ID 8:[(7, 0.972849)]
document ID 9:[(19, 0.9948625)]


In [29]:
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [30]:
n=0

print(ldamodel.get_document_topics(corpus_[n]))

print(categories[n])

print(" ".join(docs[n]))

[(3, 0.73778737), (13, 0.15116636), (17, 0.05146606), (19, 0.050901208)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesme

In [31]:
pyLDAvis.enable_notebook()

In [32]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)